# Simple Graph Convolution Network

In [1]:
# setup
from modules.data_loader import *
from modules.feature_engineering import *
from modules.visualizer import *
import networkx as nx
import dgl
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, confusion_matrix
import numpy as np

In [2]:
# #run if you dont have the data downloaded this is a few gb of data
# import kagglehub
# path = kagglehub.dataset_download("ealtman2019/ibm-transactions-for-anti-money-laundering-aml")
# print("Path to dataset files:", path)

## 1. Create our dataframes from raw data

In [3]:
dataset_name = "HI-Small"

print(f"Loading {dataset_name}...\n")
trans_df = load_transactions(dataset_size=dataset_name)
# accounts_df = load_accounts(dataset_size=dataset_name)
# patterns_df = load_patterns(dataset_size=dataset_name)
print(trans_df.head())


Loading HI-Small...


Loading transactions from: /home/linch/.cache/kagglehub/datasets/ealtman2019/ibm-transactions-for-anti-money-laundering-aml/versions/8/HI-Small_Trans.csv
File size: 453.6 MB

Loaded 5,078,345 transactions
Date range: 2022-09-01 00:00:00 to 2022-09-18 16:18:00
Laundering transactions: 5,177 (0.102%)
   transaction_id           timestamp from_bank from_account to_bank  \
0               0 2022-09-01 00:20:00       010    8000EBD30     010   
1               1 2022-09-01 00:20:00     03208    8000F4580     001   
2               2 2022-09-01 00:00:00     03209    8000F4670   03209   
3               3 2022-09-01 00:02:00       012    8000F5030     012   
4               4 2022-09-01 00:06:00       010    8000F5200     010   

  to_account  amount_received receiving_currency  amount_paid  \
0  8000EBD30          3697.34          US Dollar      3697.34   
1  8000F5340             0.01          US Dollar         0.01   
2  8000F4670         14675.57          US Dollar  

In [4]:
# convert all currencies to USD for normaliztion

trans_df = convert_currency_to_USD(trans_df)
print(trans_df.columns.tolist())

['transaction_id', 'timestamp', 'from_bank', 'from_account', 'to_bank', 'to_account', 'amount_received', 'receiving_currency', 'amount_paid', 'payment_currency', 'payment_format', 'is_laundering']


In [5]:
# compute sinusoidal temporal encodings and normalized unix timestamp

trans_df = temporal_encoding(trans_df)
print(trans_df.columns.tolist())

['transaction_id', 'timestamp', 'from_bank', 'from_account', 'to_bank', 'to_account', 'amount_received', 'receiving_currency', 'amount_paid', 'payment_currency', 'payment_format', 'is_laundering', 'hour_sin', 'hour_cos', 'time_normalized']


In [6]:
# give each currency and payment method a unique integer ID

trans_df = encode_currency_ids(trans_df)
trans_df = encode_payment_format_ids(trans_df)
print(trans_df.columns.tolist())

['transaction_id', 'timestamp', 'from_bank', 'from_account', 'to_bank', 'to_account', 'amount_received', 'receiving_currency', 'amount_paid', 'payment_currency', 'payment_format', 'is_laundering', 'hour_sin', 'hour_cos', 'time_normalized', 'payment_currency_id', 'receiving_currency_id', 'payment_format_id']


In [7]:
# give each account a unique integer ID

trans_df, account_to_id, id_to_account = encode_account_ids(trans_df)
print(trans_df.columns.tolist())

['transaction_id', 'timestamp', 'from_bank', 'from_account', 'to_bank', 'to_account', 'amount_received', 'receiving_currency', 'amount_paid', 'payment_currency', 'payment_format', 'is_laundering', 'hour_sin', 'hour_cos', 'time_normalized', 'payment_currency_id', 'receiving_currency_id', 'payment_format_id', 'from_account_id', 'to_account_id']


In [8]:
trans_df = normalize_amounts(trans_df)

## 2. Temporal train/test split and account statistics

In [9]:
# split data temporally: first 80% for training, last 20% for testing

train_df, test_df = temporal_train_test_split(trans_df, train_ratio=0.8)

import gc
del trans_df
gc.collect()


TEMPORAL TRAIN/TEST SPLIT

Train Set:
  Date range: 2022-09-01 00:00:00 to 2022-09-08 16:12:00
  Transactions: 4,062,676
  Laundering: 3,380 (0.083%)

Test Set:
  Date range: 2022-09-08 16:12:00 to 2022-09-18 16:18:00
  Transactions: 1,015,669
  Laundering: 1,797 (0.177%)


0

In [10]:
print(train_df.columns.tolist())
print(train_df.head(1))

['transaction_id', 'timestamp', 'from_bank', 'from_account', 'to_bank', 'to_account', 'amount_received', 'receiving_currency', 'amount_paid', 'payment_currency', 'payment_format', 'is_laundering', 'hour_sin', 'hour_cos', 'time_normalized', 'payment_currency_id', 'receiving_currency_id', 'payment_format_id', 'from_account_id', 'to_account_id']
   transaction_id  timestamp from_bank from_account to_bank to_account  \
0          316720 2022-09-01      0121    8123FB9B0    0121  8123FB9B0   

   amount_received receiving_currency  amount_paid payment_currency  \
0        -1.529313        Saudi Riyal    -1.529313      Saudi Riyal   

  payment_format  is_laundering  hour_sin  hour_cos  time_normalized  \
0   Reinvestment              0       0.0       1.0         -1.37763   

  payment_currency_id receiving_currency_id payment_format_id  \
0                   8                     8                 5   

   from_account_id  to_account_id  
0           458857         458857  


# Construct Graph

In [ ]:
def build_graph_from_df(df):
    source_accounts = df['from_account_id'].values
    dest_accounts = df['to_account_id'].values

    graph = dgl.graph((source_accounts, dest_accounts))
    print(f"Number of nodes: {graph.num_nodes()}")
    print(f"Number of edges: {graph.num_edges()}")
    # EDGE FEATS

    # bookkeeping
    graph.edata['transaction_id'] = torch.tensor(df['transaction_id'].values, dtype=torch.long)
    graph.edata['is_laundering'] = torch.tensor(df['is_laundering'].values, dtype=torch.long) #not used in training

    # numericals: we can include them directly
    numericals = ['amount_received', 'amount_paid', 'hour_sin', 'hour_cos', 'time_normalized']
    edge_numerical = torch.tensor(df[numericals].values, dtype=torch.float32)
    graph.edata['numericals'] = edge_numerical

    # categoricals: we use learned embeddings. This makes them transductive, but that is ok because currencies and payment types don't change often

    payment_currency = torch.tensor(df['payment_currency_id'].values, dtype=torch.long)
    receiving_currency = torch.tensor(df['receiving_currency_id'].values, dtype=torch.long)
    payment_format = torch.tensor(df['payment_format_id'].values, dtype=torch.long)
    graph.edata['payment_currency'] = payment_currency
    graph.edata['receiving_currency'] = receiving_currency
    graph.edata['payment_format'] = payment_format

    # NODE FEATS

    # For each account, we compute their in and outdegrees
    # We zero-center and log transform this value and normalize by the graph's average transformed in and outdegrees so that our model is not sensitive to the graph size. Our training graph has 80% of the edges, so naturally the raw degree counts will be higher than the testing graph.
    indegrees = graph.in_degrees().float()
    outdegrees = graph.out_degrees().float()

    log_indegrees = torch.log(indegrees + 1)
    log_outdegrees = torch.log(outdegrees + 1)

    avg_log_indegree = log_indegrees.mean()
    avg_log_outdegree = log_outdegrees.mean()

    std_log_indegree = log_indegrees.std()
    std_log_outdegree = log_outdegrees.std()

    # zc and normalize
    normalized_indegree = (log_indegrees - avg_log_indegree) / std_log_indegree
    normalized_outdegree = (log_outdegrees - avg_log_outdegree) / std_log_outdegree

    node_features = torch.stack([normalized_indegree, normalized_outdegree], dim=1)

    # Perhaps we can include simple graph RWPE as a node feature too?
    graph.ndata['node_feats'] = node_features

    return graph

In [12]:
training_graph = build_graph_from_df(train_df)
test_graph = build_graph_from_df(test_df)

Number of nodes: 515080
Number of edges: 4062676
Number of nodes: 515080
Number of edges: 1015669


In [13]:
# Check graph sanity

print("\n===== Node Features (normalized degrees) =====")
print("\n----- Node Degrees (first 10 nodes) -----")
indegrees = training_graph.in_degrees().float()
outdegrees = training_graph.out_degrees().float()

for i in range(5):
    norm_in = training_graph.ndata['node_feats'][i, 0].item()
    norm_out = training_graph.ndata['node_feats'][i, 1].item()
    print(f"Node {i}:")
    print(f"  In-degree:  {indegrees[i].item():.0f}  → normalized: {norm_in:+.4f}")
    print(f"  Out-degree: {outdegrees[i].item():.0f} → normalized: {norm_out:+.4f}")

print("\n===== Edge Features =====")
print(f"Numerical features shape: {training_graph.edata['numericals'].shape}")
print(f"Categorical feature 1 shape: {training_graph.edata['payment_currency'].shape}")
print(f"Categorical feature 2 shape: {training_graph.edata['receiving_currency'].shape}")
print(f"Categorical feature 3 shape: {training_graph.edata['payment_format'].shape}")

print("\n===== Sample Edge Features =====")
for i in range(5):
    src, dst = training_graph.edges()
    print(f"Edge {i}: {src[i].item()} -> {dst[i].item()}")
    print(f"  Numerical: {training_graph.edata['numericals'][i].tolist()}")
    print(f"  payment currency: {training_graph.edata['payment_currency'][i].item()}, receiving currency: {training_graph.edata['receiving_currency'][i].item()} , payment format: {training_graph.edata['payment_format'][i].item()}")


===== Node Features (normalized degrees) =====

----- Node Degrees (first 10 nodes) -----
Node 0:
  In-degree:  545  → normalized: +3.1889
  Out-degree: 132597 → normalized: +7.6085
Node 1:
  In-degree:  328  → normalized: +2.8523
  Out-degree: 80883 → normalized: +7.2477
Node 2:
  In-degree:  55  → normalized: +1.6754
  Out-degree: 14711 → normalized: +6.0038
Node 3:
  In-degree:  53  → normalized: +1.6512
  Out-degree: 10810 → normalized: +5.7789
Node 4:
  In-degree:  61  → normalized: +1.7430
  Out-degree: 13641 → normalized: +5.9487

===== Edge Features =====
Numerical features shape: torch.Size([4062676, 5])
Categorical feature 1 shape: torch.Size([4062676])
Categorical feature 2 shape: torch.Size([4062676])
Categorical feature 3 shape: torch.Size([4062676])

===== Sample Edge Features =====
Edge 0: 458857 -> 458857
  Numerical: [-1.5293132066726685, -1.529313087463379, 0.0, 1.0, -1.377630352973938]
  payment currency: 8, receiving currency: 8 , payment format: 5
Edge 1: 236631 -

# Define model architecture

In [14]:
# since we have both categorical and numerical edge features, we need a class to process and combine them

class EdgeEmbedding(nn.Module):
    def __init__(
            self, 
            num_currencies=15, 
            num_payment_methods=7, 
            currencies_embed_dim=8, 
            payment_embed_dim=4, 
            num_numericals=5):
        super().__init__()
        self.currencies_embed = nn.Embedding(num_currencies, currencies_embed_dim)
        self.payment_embed = nn.Embedding(num_payment_methods, payment_embed_dim)

        # each edge has payment currency, receiving currency, payment method and numericals
        self.out_dim = currencies_embed_dim + currencies_embed_dim + payment_embed_dim + num_numericals

    def forward(self, payment_curr, receiving_curr, payment_method, numericals):
        payment_curr_embed = self.currencies_embed(payment_curr)
        receiving_curr_embed = self.currencies_embed(receiving_curr)
        payment_method_embed = self.payment_embed(payment_method)

        edge_feats = torch.cat([payment_curr_embed, receiving_curr_embed, payment_method_embed, numericals], dim=1)
        return edge_feats

# this returns our GNN-ready edge features

In [15]:
# message passing layers

class GCNLayer(nn.Module):
    def __init__(self, input_dim, output_dim, residual=True):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.residual = residual

        if residual and input_dim != output_dim:
            self.res_linear = nn.Linear(input_dim, output_dim) # project residual if diff dims
        elif residual:
            self.res_linear = nn.Identity()

    def message_func(self, edges):
        return {'m': edges.src['Wh'] * edges.src['norm']} # normalize by source degree first

    def reduce_func(self, nodes):
        return {'h': torch.sum(nodes.mailbox['m'], dim=1)}
    
    def forward(self, graph, node_feats):
        with graph.local_scope():
            h_in = node_feats

            graph.ndata['Wh'] = self.linear(node_feats) # W is linear transform, h is node feats

            degs = graph.in_degrees().float().clamp(min=1)
            norm = torch.pow(degs, -0.5).unsqueeze(1)
            graph.ndata['norm'] = norm

            graph.update_all(self.message_func, self.reduce_func)

            h_out = graph.ndata['h'] * norm # then normalize by destination degree

            h_out = F.relu(h_out)
            if self.residual:
                return h_out + self.res_linear(h_in)
            else:
                return h_out



In [16]:
# Full classifier neural network

class GCNEdgeClassifier(nn.Module):
    
    def __init__(self,
                # node params
                node_in_feats=2, # log transformed and normalized indegree outdegree
                hidden_dim=64,
                num_gcn_layers=3,

                # for EdgeEmbedding
                num_currencies=15,
                num_payment_methods=7,
                currencies_embed_dim=8,
                payment_embed_dim=4,
                num_numericals=5,

                # output classes
                num_classes=2,

                # regularization
                dropout=0.2,
                use_batch_norm=False):
        super().__init__()

        self.num_gcn_layers = num_gcn_layers
        self.hidden_dim = hidden_dim
        self.dropout = dropout
        self.use_batch_norm = use_batch_norm

        # GCN layers
        self.gcn_layers = nn.ModuleList()
        self.gcn_layers.append(GCNLayer(node_in_feats, hidden_dim, residual=False)) # no residual for layer 1 since in dim and out dim dont match, simpler to just skip
        for i in range(num_gcn_layers - 1):
            self.gcn_layers.append(GCNLayer(hidden_dim, hidden_dim, residual=True))
        if use_batch_norm: # create batch norm layers if using them
            self.batch_norms = nn.ModuleList([nn.BatchNorm1d(hidden_dim) for i in range(num_gcn_layers)])

        # Edge Feature processor
        self.edge_embed = EdgeEmbedding(num_currencies, num_payment_methods, currencies_embed_dim, payment_embed_dim, num_numericals)

        # Combiner to do edge classification: append 2 node embeddings with edge embedding
        edge_repr_dim = 2*hidden_dim + self.edge_embed.out_dim

        # MLP for edge classification
        self.edge_classifier = nn.Sequential(
            nn.Linear(edge_repr_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim) if use_batch_norm else nn.Identity(),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim) if use_batch_norm else nn.Identity(),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, num_classes)
        )


    def forward(self, graph):
        node_features = graph.ndata['node_feats']
        payment_currency = graph.edata['payment_currency']
        receiving_currency = graph.edata['receiving_currency']
        payment_method = graph.edata['payment_format']
        edge_numericals = graph.edata['numericals']

        # GCN message passing to learn node embeddings
        h = node_features
        for i, gcn_layer in enumerate(self.gcn_layers):
            h = gcn_layer(graph, h)
            if self.use_batch_norm:
                h = self.batch_norms[i](h)
            if i < self.num_gcn_layers - 1: # dropout between each layer except after last
                h = F.dropout(h, p=self.dropout, training=self.training)
        
        # get node embedding for each edge
        src_nodes, dst_nodes = graph.edges()
        src_embed = h[src_nodes]
        dst_embed = h[dst_nodes]

        # process edge features
        edge_features = self.edge_embed(payment_currency, receiving_currency, payment_method, edge_numericals)

        # concat 2 nodes embeddings with edge features to get complete edge embedding
        edge_repr = torch.cat([src_embed, dst_embed, edge_features], dim=1)

        # classify edges
        logits = self.edge_classifier(edge_repr)

        return logits

In [17]:
# Define focal CE loss since we have extreme class imbalance
# NOT IN USE CURRENTLY. Didn't work very well. Current implementation uses weighted normal CE

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, logits, targets):
        probs = F.softmax(logits, dim=1)
        targets_one_hot = F.one_hot(targets, num_classes=logits.shape[1])
        p_t = (probs * targets_one_hot).sum(dim=1) # probability of true class in dataset
        focal_weight = (1 - p_t) ** self.gamma # easy examples p_t near 1, focal weight near 0

        ce_loss = F.cross_entropy(logits, targets, reduction='none')
        focal_loss = focal_weight * ce_loss

        if self.alpha is not None:
            alpha_t = self.alpha[targets]  # [N]
            focal_loss = alpha_t * focal_loss

        if self.reduction == 'mean':
            return focal_loss.mean() #default
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [18]:
def evaluate(model, graph):
    """Evaluate model performance."""
    model.eval()
    
    with torch.no_grad():
        # Pass edge features as separate arguments from graph.edata
        logits = model(graph)
        
        labels = graph.edata['is_laundering']
        preds = logits.argmax(dim=1)
        probs = torch.softmax(logits, dim=1)[:, 1]
        
        accuracy = (preds == labels).float().mean().item()
        
        precision, recall, f1, support = precision_recall_fscore_support(
            labels.cpu().numpy(), 
            preds.cpu().numpy(),
            average=None,
            zero_division=0
        )
        
        try:
            auc = roc_auc_score(labels.cpu().numpy(), probs.cpu().numpy())
        except:
            auc = 0.0
        
        cm = confusion_matrix(labels.cpu().numpy(), preds.cpu().numpy())
        
        metrics = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'support': support,
            'auc': auc,
            'confusion_matrix': cm
        }
        
        return metrics, preds, probs

# Training

In [ ]:
# Setup model and loss

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nUsing device: {device}")

training_graph = training_graph.to(device)
model = GCNEdgeClassifier().to(device)

train_labels = training_graph.edata['is_laundering'].long()

############ MINORITY CLASS SCALE WEIGHT ############
class_weights = torch.tensor([1.0, 20.0]).to(device) #place 20x weight on positive examples, since we are doing 1:30 undersampling, feel free to adjust these numbers and experiment yourself
######################################################

print(f"Class weights: {class_weights}")
# criterion = FocalLoss(alpha=class_weights, gamma=1.5)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# support code for loss level undersampling
train_labels = training_graph.edata['is_laundering'].long()
class_0_idx = (train_labels == 0).nonzero(as_tuple=True)[0]
class_1_idx = (train_labels == 1).nonzero(as_tuple=True)[0]
n_class_1 = len(class_1_idx)
n_class_0_sample = n_class_1 * 30 # We want to sample 30 legitimate transactions for every laundering transaction during loss calc


Using device: cuda


Class weights: tensor([ 1., 20.], device='cuda:0')


In [20]:
num_epochs = 50
best_f1 = 0.0

print("\n" + "="*60)
print("TRAINING")
print("="*60)

for epoch in range(num_epochs):

    # perform the random subsampling of legitimate transactions every epoch
    class_0_sampled = class_0_idx[torch.randperm(len(class_0_idx))[:n_class_0_sample]]
    train_idx = torch.cat([class_0_sampled, class_1_idx])
    train_mask = torch.zeros(len(train_labels), dtype=torch.bool)
    train_mask[train_idx] = True

    # Train
    model.train()
    optimizer.zero_grad()
    
    logits = model(training_graph)
    loss = criterion(logits[train_mask], train_labels[train_mask]) # apply the mask that subsamples legitimate transactions when calculating loss
    
    loss.backward()
    optimizer.step()
    
    # Evaluate every 2 epochs, can change to 5 if uw, this slows down training 
    if epoch % 2 == 0:
        model.eval()
        with torch.no_grad():
            eval_logits = model(training_graph)
            preds = eval_logits.argmax(dim=1)
            
            acc = (preds == train_labels).float().mean().item()
            
            # Class 1 metrics
            tp = ((preds == 1) & (train_labels == 1)).sum().item()
            fp = ((preds == 1) & (train_labels == 0)).sum().item()
            fn = ((preds == 0) & (train_labels == 1)).sum().item()
            
            precision_1 = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall_1 = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1) if (precision_1 + recall_1) > 0 else 0
            
            pred_1_pct = (preds == 1).float().mean().item() * 100
            
        print(f"\nEpoch {epoch:3d} | Loss: {loss.item():.4f} | Acc: {acc:.4f}")
        print(f"  Class 1 - P: {precision_1:.4f}, R: {recall_1:.4f}, F1: {f1_1:.4f}")
        print(f"  Predicting class 1: {pred_1_pct:.5f}%")
        
        # Save best model
        if f1_1 > best_f1:
            best_f1 = f1_1
            torch.save(model.state_dict(), 'best_gcn_model.pt')
            print(f"  Saved best model (F1: {best_f1:.4f})")
        
        del eval_logits, preds
        torch.cuda.empty_cache()
    else:
        print(f"Epoch {epoch:3d} | Loss: {loss.item():.4f}")
    
    del logits
    torch.cuda.empty_cache()

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)


TRAINING

Epoch   0 | Loss: 0.6830 | Acc: 0.9636
  Class 1 - P: 0.0014, R: 0.0612, F1: 0.0028
  Predicting class 1: 3.56376%
  Saved best model (F1: 0.0028)
Epoch   1 | Loss: 0.6638

Epoch   2 | Loss: 0.6548 | Acc: 0.9689
  Class 1 - P: 0.0014, R: 0.0527, F1: 0.0028
  Predicting class 1: 3.03101%
  Saved best model (F1: 0.0028)
Epoch   3 | Loss: 0.6462

Epoch   4 | Loss: 0.6384 | Acc: 0.9475
  Class 1 - P: 0.0015, R: 0.0917, F1: 0.0029
  Predicting class 1: 5.18286%
  Saved best model (F1: 0.0029)
Epoch   5 | Loss: 0.6343

Epoch   6 | Loss: 0.6288 | Acc: 0.9414
  Class 1 - P: 0.0016, R: 0.1095, F1: 0.0031
  Predicting class 1: 5.79660%
  Saved best model (F1: 0.0031)
Epoch   7 | Loss: 0.6241

Epoch   8 | Loss: 0.6139 | Acc: 0.9193
  Class 1 - P: 0.0025, R: 0.2373, F1: 0.0049
  Predicting class 1: 8.02594%
  Saved best model (F1: 0.0049)
Epoch   9 | Loss: 0.6135

Epoch  10 | Loss: 0.6060 | Acc: 0.8988
  Class 1 - P: 0.0031, R: 0.3749, F1: 0.0061
  Predicting class 1: 10.09509%
  Saved 

In [21]:
# run this if your gpu isnt cash money 
print("\nCleaning up training graph...")
training_graph = training_graph.to('cpu')
del training_graph, train_labels
torch.cuda.empty_cache()



Cleaning up training graph...


In [22]:

# Load best model
print("Loading best model...")
model.load_state_dict(torch.load('best_gcn_model.pt'))
test_graph = test_graph.to(device)

test_labels = test_graph.edata['is_laundering'].long()

print(f"Class 0: {(test_labels==0).sum()} ({100*(test_labels==0).float().mean():.2f}%)")
print(f"Class 1: {(test_labels==1).sum()} ({100*(test_labels==1).float().mean():.2f}%)")

# Evaluate
model.eval()
with torch.no_grad():
    test_logits = model(test_graph)
    test_preds = test_logits.argmax(dim=1)
    test_probs = torch.softmax(test_logits, dim=1)[:, 1]
    
    # Metrics
    acc = (test_preds == test_labels).float().mean().item()
    
    tp = ((test_preds == 1) & (test_labels == 1)).sum().item()
    fp = ((test_preds == 1) & (test_labels == 0)).sum().item()
    fn = ((test_preds == 0) & (test_labels == 1)).sum().item()
    tn = ((test_preds == 0) & (test_labels == 0)).sum().item()
    
    precision_0 = tn / (tn + fn) if (tn + fn) > 0 else 0
    recall_0 = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    precision_1 = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall_1 = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1) if (precision_1 + recall_1) > 0 else 0
    
    try:
        auc = roc_auc_score(test_labels.cpu().numpy(), test_probs.cpu().numpy())
    except:
        auc = 0.0

print("\n" + "="*60)
print("TEST SET RESULTS")
print("="*60)
print(f"Overall Accuracy: {acc:.4f}")
print(f"AUC-ROC: {auc:.4f}")
print(f"\nConfusion Matrix:")
print(f"  TN: {tn:7d}  FP: {fp:7d}")
print(f"  FN: {fn:7d}  TP: {tp:7d}")
print(f"\nClass 0 (Legitimate):")
print(f"  Precision: {precision_0:.4f}")
print(f"  Recall: {recall_0:.4f}")
print(f"\nClass 1 (Laundering):")
print(f"  Precision: {precision_1:.4f}")
print(f"  Recall: {recall_1:.4f}")
print(f"  F1 Score: {f1_1:.4f}")
print("="*60)

Loading best model...
Class 0: 1013872 (99.82%)
Class 1: 1797 (0.18%)

TEST SET RESULTS
Overall Accuracy: 0.9316
AUC-ROC: 0.8532

Confusion Matrix:
  TN:  945291  FP:   68581
  FN:     873  TP:     924

Class 0 (Legitimate):
  Precision: 0.9991
  Recall: 0.9324

Class 1 (Laundering):
  Precision: 0.0133
  Recall: 0.5142
  F1 Score: 0.0259
